In [1]:
# Olist E-Commerce - Initial Exploration
# Phase 1: Data Discovery and Business Understanding

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add project root to path to import data_ingestion module
project_root = Path('../').resolve()
sys.path.append(str(project_root))

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')

print("=== OLIST COMBINED DATASET EXPLORATION ===")
print("Goal: Understand data structure, relationships, and business opportunities")
print("Datasets: Brazilian E-Commerce + Marketing Funnel")
print("Data location: Centralized data folder at project root")
print("\n" + "="*60 + "\n")


=== OLIST COMBINED DATASET EXPLORATION ===
Goal: Understand data structure, relationships, and business opportunities
Datasets: Brazilian E-Commerce + Marketing Funnel
Data location: Centralized data folder at project root




In [2]:
# =============================================================================
# DATA SETUP: USE CENTRALIZED DATA INGESTION
# =============================================================================

print("=== DATA SETUP USING CENTRALIZED INGESTION ===")
print("Using data_ingestion.py script for consistent data management...")
print("-" * 60)

# Import data ingestion functions
try:
    from data_ingestion import setup_kaggle_datasets, load_datasets
    print("✓ Data ingestion module imported successfully")
except ImportError:
    print("❌ Error: Could not import data_ingestion module")
    print("Please ensure data_ingestion.py exists in the notebooks directory")
    print("Run the following command from notebooks directory:")
    print("python data_ingestion.py")
    
# Option 1: Run full data ingestion if needed
# This will download datasets if not already present
print("\n--- Checking if datasets need to be downloaded ---")
data_dir = Path('../data')
if not data_dir.exists() or len(list(data_dir.rglob("*.csv"))) < 9:
    print("📥 Running data ingestion (download + setup)...")
    setup_success = setup_kaggle_datasets()
    if not setup_success:
        print("❌ Data setup failed. Please run data_ingestion.py manually from notebooks directory")
else:
    print("✓ Data directory exists with expected files")

print("\n" + "="*50)
print("✅ DATA SETUP COMPLETE!")
print("Data is ready for loading and analysis.")
print("="*50)

=== DATA SETUP USING CENTRALIZED INGESTION ===
Using data_ingestion.py script for consistent data management...
------------------------------------------------------------
✓ Data ingestion module imported successfully

--- Checking if datasets need to be downloaded ---
✓ Data directory exists with expected files

✅ DATA SETUP COMPLETE!
Data is ready for loading and analysis.


In [3]:
# =============================================================================
# STEP 1: DATA INVENTORY AND LOADING FROM CENTRALIZED DATA FOLDER
# =============================================================================

print("STEP 1: DATA INVENTORY AND LOADING")
print("Loading from centralized data folder at project root...")
print("-" * 50)

# Define file paths - using centralized data folder
ecommerce_path = Path('../data/')

# Verify data folder exists
if not ecommerce_path.exists():
    print("❌ Error: Data folder not found!")
    print("Please run the data_ingestion.py script first:")
    print("cd notebooks && python data_ingestion.py")
else:
    print(f"✓ Data folder found: {ecommerce_path}")

# Use the load_datasets function from data_ingestion module
print("\n--- Using centralized data loading function ---")
try:
    # Load datasets using the centralized function
    ecommerce_data = load_datasets()
    
    # Display summary
    print(f"\n📊 DATA LOADING SUMMARY:")
    print(f"E-Commerce datasets: {len(ecommerce_data)} tables loaded")
    
except Exception as e:
    print(f"❌ Error using centralized loading: {e}")
    print("Falling back to manual loading...")
    
    # Fallback: Manual loading
    # E-Commerce dataset files
    ecommerce_files = {
        'customers': 'olist_customers_dataset.csv',
        'orders': 'olist_orders_dataset.csv', 
        'order_items': 'olist_order_items_dataset.csv',
        'order_payments': 'olist_order_payments_dataset.csv',
        'order_reviews': 'olist_order_reviews_dataset.csv',
        'products': 'olist_products_dataset.csv',
        'sellers': 'olist_sellers_dataset.csv',
        'geolocation': 'olist_geolocation_dataset.csv',
        'category_translation': 'product_category_name_translation.csv'
    }
    
    # Load all datasets manually
    print("Loading E-Commerce datasets...")
    ecommerce_data = {}
    for name, filename in ecommerce_files.items():
        try:
            df = pd.read_csv(ecommerce_path / filename)
            ecommerce_data[name] = df
            print(f"✓ {name}: {df.shape[0]:,} rows x {df.shape[1]} cols")
        except FileNotFoundError:
            print(f"✗ {name}: File not found - {filename}")
        except Exception as e:
            print(f"✗ {name}: Error loading - {e}")


STEP 1: DATA INVENTORY AND LOADING
Loading from centralized data folder at project root...
--------------------------------------------------
✓ Data folder found: ../data

--- Using centralized data loading function ---

=== LOADING DATASETS ===
Loading E-Commerce dataset into memory...
----------------------------------------
Loading E-Commerce datasets...
✓ customers: 99,441 rows x 5 cols
✓ orders: 99,441 rows x 8 cols
✓ order_items: 112,650 rows x 7 cols
✓ order_payments: 103,886 rows x 5 cols
✓ order_reviews: 99,224 rows x 7 cols
✓ products: 32,951 rows x 9 cols
✓ sellers: 3,095 rows x 4 cols
✓ geolocation: 1,000,163 rows x 5 cols
✓ category_translation: 71 rows x 2 cols

✅ Data loading complete!
E-Commerce datasets loaded: 9

📊 DATA LOADING SUMMARY:
E-Commerce datasets: 9 tables loaded


In [4]:
# =============================================================================
# STEP 2: INITIAL DATA PROFILING
# =============================================================================

print("\n" + "="*60)
print("STEP 2: INITIAL DATA PROFILING")

def profile_dataset(df, name):
    """Quick data profiling function"""
    print(f"\n=== {name.upper()} PROFILE ===")
    print(f"Shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Display columns as bulleted list
    print(f"Columns ({len(df.columns)} total):")
    for col in df.columns:
        print(f"  • {col}")
    
    # Null values
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"Null values:")
        for col, count in null_counts[null_counts > 0].items():
            print(f"  {col}: {count:,} ({count/len(df)*100:.1f}%)")
    else:
        print("No null values found")
    
    # Data types
    print(f"Data types: {df.dtypes.value_counts().to_dict()}")
    
    return df.describe(include='all')

# Profile ALL E-commerce datasets
print("\n=== PROFILING ALL E-COMMERCE DATASETS ===")
ecommerce_profiles = {}
for dataset_name, dataset_df in ecommerce_data.items():
    print(f"\n{'='*60}")
    ecommerce_profiles[dataset_name] = profile_dataset(dataset_df, f"E-Commerce: {dataset_name}")

print(f"\n{'='*60}")
print(f"SUMMARY: Profiled {len(ecommerce_profiles)} e-commerce datasets")
print(f"E-commerce datasets: {list(ecommerce_profiles.keys())}")


STEP 2: INITIAL DATA PROFILING

=== PROFILING ALL E-COMMERCE DATASETS ===


=== E-COMMERCE: CUSTOMERS PROFILE ===
Shape: (99441, 5)
Memory usage: 29.62 MB
Columns (5 total):
  • customer_id
  • customer_unique_id
  • customer_zip_code_prefix
  • customer_city
  • customer_state
No null values found
Data types: {dtype('O'): 4, dtype('int64'): 1}


=== E-COMMERCE: ORDERS PROFILE ===
Shape: (99441, 8)
Memory usage: 58.97 MB
Columns (8 total):
  • order_id
  • customer_id
  • order_status
  • order_purchase_timestamp
  • order_approved_at
  • order_delivered_carrier_date
  • order_delivered_customer_date
  • order_estimated_delivery_date
Null values:
  order_approved_at: 160 (0.2%)
  order_delivered_carrier_date: 1,783 (1.8%)
  order_delivered_customer_date: 2,965 (3.0%)
Data types: {dtype('O'): 8}


=== E-COMMERCE: ORDER_ITEMS PROFILE ===
Shape: (112650, 7)
Memory usage: 39.43 MB
Columns (7 total):
  • order_id
  • order_item_id
  • product_id
  • seller_id
  • shipping_limit_date
  • pr

In [10]:
# =============================================================================
# STEP 3: E-COMMERCE DATA RELATIONSHIP ANALYSIS
# =============================================================================

print("\n" + "="*60)
print("STEP 3: E-COMMERCE DATA RELATIONSHIP ANALYSIS")
print("-" * 40)

# Analyze key relationships in the e-commerce data
if 'orders' in ecommerce_data and 'customers' in ecommerce_data:
    orders_df = ecommerce_data['orders']
    customers_df = ecommerce_data['customers']
    
    print("E-Commerce Data Structure:")
    print(f"1. Total Orders: {orders_df.shape[0]:,}")
    
    # Based on Kaggle documentation
    # customer_id: key to orders dataset (each order has unique customer_id)
    # customer_unique_id: unique identifier of actual customers
    total_unique_customers = customers_df['customer_unique_id'].nunique()
    
    # To calculate customers with orders, we need to map customer_id to customer_unique_id
    # Create mapping from customer_id to customer_unique_id
    customer_mapping = customers_df[['customer_id', 'customer_unique_id']].set_index('customer_id')['customer_unique_id']
    
    # Map orders to actual unique customers
    orders_with_unique_customers = orders_df.copy()
    orders_with_unique_customers['customer_unique_id'] = orders_with_unique_customers['customer_id'].map(customer_mapping)
    
    # Count unique customers who have orders
    customers_with_orders = orders_with_unique_customers['customer_unique_id'].nunique()
    
    print(f"2. Total Unique Customers: {total_unique_customers:,}")
    print(f"3. Total Customers with Orders: {customers_with_orders:,}")
    
    # Analyze customer order frequency based on actual unique customers
    unique_customer_order_counts = orders_with_unique_customers.groupby('customer_unique_id').size()
    repeat_customers = (unique_customer_order_counts > 1).sum()
    one_time_customers = (unique_customer_order_counts == 1).sum()
    
    print(f"\n4. Customer Purchase Behavior (Based on Actual Unique Customers):")
    print(f"   - One-time customers: {one_time_customers:,} ({one_time_customers/customers_with_orders*100:.1f}%)")
    print(f"   - Repeat customers: {repeat_customers:,} ({repeat_customers/customers_with_orders*100:.1f}%)")
    print(f"   - Max orders per unique customer: {unique_customer_order_counts.max()}")
    print(f"   - Average orders per unique customer: {unique_customer_order_counts.mean():.2f}")
    
    # Order status distribution
    if 'order_status' in orders_df.columns:
        print(f"\n5. Order Status Distribution:")
        status_counts = orders_df['order_status'].value_counts()
        for status, count in status_counts.items():
            percentage = count / len(orders_df) * 100
            print(f"   - {status}: {count:,} ({percentage:.1f}%)")

# Check seller information
if 'sellers' in ecommerce_data and 'order_items' in ecommerce_data:
    sellers_df = ecommerce_data['sellers']
    order_items_df = ecommerce_data['order_items']
    
    total_sellers = sellers_df['seller_id'].nunique()
    active_sellers = order_items_df['seller_id'].nunique()
    print(f"\n6. Seller Activity:")
    print(f"   - Total Sellers: {total_sellers:,}")
    print(f"   - Active Sellers (with orders): {active_sellers:,}")
    if total_sellers > 0:
        activity_rate = (active_sellers / total_sellers * 100)
        print(f"   - Seller Activity Rate: {activity_rate:.1f}%")

# Geographic distribution
if 'customers' in ecommerce_data:
    print(f"\n7. Customer Geographic Distribution (Top 10 States):")
    customer_state_counts = ecommerce_data['customers']['customer_state'].value_counts().head(10)
    for state, count in customer_state_counts.items():
        percentage = count / len(ecommerce_data['customers']) * 100
        print(f"   - {state}: {count:,} ({percentage:.1f}%)")


STEP 3: E-COMMERCE DATA RELATIONSHIP ANALYSIS
----------------------------------------
E-Commerce Data Structure:
1. Total Orders: 99,441
2. Total Unique Customers: 96,096
3. Total Customers with Orders: 96,096

4. Customer Purchase Behavior (Based on Actual Unique Customers):
   - One-time customers: 93,099 (96.9%)
   - Repeat customers: 2,997 (3.1%)
   - Max orders per unique customer: 17
   - Average orders per unique customer: 1.03

5. Order Status Distribution:
   - delivered: 96,478 (97.0%)
   - shipped: 1,107 (1.1%)
   - canceled: 625 (0.6%)
   - unavailable: 609 (0.6%)
   - invoiced: 314 (0.3%)
   - processing: 301 (0.3%)
   - created: 5 (0.0%)
   - approved: 2 (0.0%)

6. Seller Activity:
   - Total Sellers: 3,095
   - Active Sellers (with orders): 3,095
   - Seller Activity Rate: 100.0%

7. Customer Geographic Distribution (Top 10 States):
   - SP: 41,746 (42.0%)
   - RJ: 12,852 (12.9%)
   - MG: 11,635 (11.7%)
   - RS: 5,466 (5.5%)
   - PR: 5,045 (5.1%)
   - SC: 3,637 (3.7%)
